<a href="https://colab.research.google.com/github/pgordin/OptDisc2025/blob/main/Lab1_LP_ILP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linki do tutoriali

[Programowanie liniowe](https://medium.com/@chongjingting/4-ways-to-solve-linear-programming-in-python-b4af36b7894d)

[Programowanie całkowitoliczbowe w PULP](https://medium.com/@gazalashaikh999/mixed-integer-programming-cfe0c196e875)

[Automatyzacja programowania całkowitoliczbowego w Google-OR
](https://developers.google.com/optimization/mip/mip_var_array?hl=pl)

# Zadanie 1.
Na przykładzie ciągłego problemu plecakowego sprawdzić (i przetestować) SciPy, PuLP i Google OR-Tools.

**1. ScipPy**

In [76]:
import numpy as np
from scipy.optimize import linprog

Dane dla przedmiotów

In [77]:
#wartości przedmiotów
values = [60, 100, 120]

#wagi przedmiotów
weights = [10, 20, 30]

#maksymalna waga plecaka
W = 50

#liczba przedmiotów
n = len(values)

Funkcja celu

In [78]:
c = [-v for v in values]

Wprowadzone ograniczenia

In [79]:
#ograniczenie sumy wag
A = [weights]

#ograniczenie na maksymalną wagę plecaka
b = [W]

# ograniczenie, że x_i jest w przedziale [0, 1]
bounds = [(0, 1) for _ in range(n)]

Rozwiązanie problemu

In [80]:
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
if result.success:
    print("Optymalne rozwiązanie")
    print(f"Wybrane przedmioty: {np.round(result.x, 2)}")
    print(f"Łączna wartość plecaka: {-result.fun}")
else:
    print("Nie znaleziono rozwiązania")

Optymalne rozwiązanie
Wybrane przedmioty: [1.   1.   0.67]
Łączna wartość plecaka: 240.0


**2. PuLP**

In [81]:
!pip install pulp

In [82]:
from pulp import *

In [83]:
model = pulp.LpProblem('linear_programming', LpMaximize)

get solver

In [84]:
solver = getSolver('PULP_CBC_CMD')

Zmienne decyzyjne

In [85]:
# ilosci wzietych poszczegolnych produktów
x1 = LpVariable('x1', lowBound = 0, cat = 'Continuous')
x2 = LpVariable('x2', lowBound = 0, cat = 'Continuous')
x3 = LpVariable('x3', lowBound = 0, cat = 'Continuous')
x4 = LpVariable('x4', lowBound = 0, cat = 'Continuous')
x5 = LpVariable('x5', lowBound = 0, cat = 'Continuous')

#pojemnosc plecaka
pojemnosc = 1200

#maksymalne możliwe ilości do wzięcia
quant = [10,7,8,5,6]

#masy jednostkowe produktów
mass = [46, 40, 42, 38, 10]

#ceny jednostkowe produktów
values = [12, 19, 19, 15, 8]

Cel

In [86]:
model += values[0]*x1+values[1]*x2+values[2]*x3+values[3]*x4+values[4]*x5

Wprowadzenie ograniczeń

In [87]:
model += mass[0]*x1+mass[1]*x2+mass[2]*x3+mass[3]*x4+mass[4]*x5 <= pojemnosc

for i in range(1,6):
    model += eval('x{}'.format(i)) <= quant[i-1]

Rozwiązanie

In [88]:
results = model.solve(solver=solver)

# print results
if LpStatus[results] == 'Optimal': print('The solution is optimal.')
print(f'Objective value: z* = {value(model.objective)}')
print(f'Mass value: z* = {mass[0]*value(x1)+mass[1]*value(x2)+mass[2]*value(x3)+mass[3]*value(x4)+mass[4]*value(x5)}')
print(f'Solution: x1* = {value(x1)}, x2* = {value(x2)}, x3* = {value(x3)}, x4* = {value(x4)}, x5* = {value(x5)}')

The solution is optimal.
Objective value: z* = 495.1304352
Mass value: z* = 1200.0000016
Solution: x1* = 7.2608696, x2* = 7.0, x3* = 8.0, x4* = 5.0, x5* = 6.0


**3. Google OR-Tools**

Podstawy - problem plecakowy w wersji ciągłej

In [89]:
!pip install ortools

In [90]:
from ortools.linear_solver import pywraplp

#ustalenie stałych
#wartości przedmiotów:
item1value = 100
item2value = 0.5

#pojemność plecaka:
knapsack_cap = 20

#dostępna masa przedmiotów:
item1maxmass = 6.3
item2maxmass = 15

# wywołanie solvera
solver = pywraplp.Solver.CreateSolver('GLOP')

# określenie zmiennych
item1mass = solver.NumVar(0.0, solver.infinity(), 'item1mass')
item2mass = solver.NumVar(0.0, solver.infinity(), 'item2mass')

# określenie celu
solver.Maximize(item1value*item1mass + item2value*item2mass)

# określenie ograniczeń
solver.Add(item1mass + item2mass <= knapsack_cap)
solver.Add(item1mass <= item1maxmass)
solver.Add(item2mass <= item2maxmass)

# rozwiązywanie
results = solver.Solve()

# wyświetlenie wyniku:
if results == pywraplp.Solver.OPTIMAL: print(f'The solution is optimal.')
print(f'Objective value: z* = {solver.Objective().Value()}')
print(f'Solution: item1mass* = {item1mass.solution_value()}, item2mass* = {item2mass.solution_value()}')

The solution is optimal.
Objective value: z* = 636.85
Solution: item1mass* = 6.3, item2mass* = 13.700000000000001


Funkcja do rozwiązywania problemu plecakowego w wersji ciągłej, z jednym plecakiem

In [91]:
import numpy as np
def solving_knapsack_problem(itemvalues, itemmass, itemBounds, capacity):

    solver = pywraplp.Solver.CreateSolver('SCIP')

    # określanie zmiennych (iteracyjnie, w zależności od tego co podamy do funkcji)

    itemCounts = np.array([solver.IntVar(0.0, solver.infinity(), f'item{i + 1}Count') for i in range(len(itemvalues))])

    # określanie celu
    solver.Maximize(sum(itemvalues*itemCounts))

    # określanie ograniczeń (iteracyjnie, w zależności od tego co podamy do funkcji)

    for i in range(len(itemBounds)):
        solver.Add(itemCounts[i] <= itemBounds[i])


    solver.Add(solver.Sum(itemCounts[i] * itemmass[i] for i in range(len(itemmass))) <= capacity)


    # rozwiązanie
    results = solver.Solve()

    # wyświetlanie wyników
    if results == pywraplp.Solver.OPTIMAL: print(f'The solution is optimal.')
    print(f'Objective value: z* = {solver.Objective().Value()}')
    for i in range(len(itemCounts)):
        print(f'Solution: item{i+1}Count* = {itemCounts[i].solution_value()}')

Przykładowe wykorzystanie

In [92]:
# przykładowe dane

itemvalues = np.array([100, 0.5])
itemmass = np.array([16.3, 2.2])
itemCounts = []
itemBounds = np.array([6, 15])
capacity = 95

solving_knapsack_problem(itemvalues, itemmass, itemBounds, capacity)

The solution is optimal.
Objective value: z* = 502.99999999999994
Solution: item1Count* = 5.0
Solution: item2Count* = 6.0


- - -

# Zadanie 2.
Jak wygląda programowanie całkowitoliczbowe w tych pakietach? Rozwiązać (zwykły) problem plecakowy.


- - -

**1. ScipPy**

SciPy nie obsługuje programowania całkowitoliczbowego bezpośrednio, więc do rozwiązania tego typu problemu możemy sprawdzić wszystkie kombinacje wybranych przedmiotów i znaleźć najlepszą możliwą całkowitoliczbową wartość.

In [93]:
import numpy as np
from scipy.optimize import linprog
from itertools import combinations

Dane dla przedmiotów

In [94]:
#wartości przedmiotów
values = [60, 100, 120]

#wagi przedmiotów
weights = [10, 20, 30]

#maksymalna waga plecaka
W = 50

#liczba przedmiotów
n = len(values)

Funkcja celu

In [95]:
c = [-v for v in values]

Ograniczenia dla funkcji

In [96]:
#ograniczenie wagowe
A = [weights]

#ograniczenie na maksymalną wagę plecaka
b = [W]

#ograniczenia x_i ∈ {0, 1}
bounds = [(0, 1) for _ in range(n)]

Rozwiązanie problemu

In [97]:
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
if result.success:
    #zaokrąglamy i sprawdzamy wszystkie kombinacje
    best_value = 0
    best_combination = None

    #przegląd wszystkich kombinacji przedmiotów
    for r in range(1, n + 1):
        for subset in combinations(range(n), r):
            subset_weight = sum(weights[i] for i in subset)
            subset_value = sum(values[i] for i in subset)

            if subset_weight <= W and subset_value > best_value:
                best_value = subset_value
                best_combination = subset

    selected_items = [1 if i in best_combination else 0 for i in range(n)]

    print("Optymalne rozwiązanie")
    print(f"Wybrane przedmioty: {selected_items}")
    print(f"Łączna wartość plecaka: {best_value}")
else:
    print("Nie znaleziono rozwiązania")

Optymalne rozwiązanie
Wybrane przedmioty: [0, 1, 1]
Łączna wartość plecaka: 220


**2. PuLP**

In [98]:
!pip install pulp

In [99]:
from pulp import *

model = pulp.LpProblem('linear_programming', LpMaximize)

# get solver
solver = getSolver('PULP_CBC_CMD')

# declare decision variables - ilosci wzietych poszczegolnych produktów
x1 = LpVariable('x1', lowBound = 0, cat = 'Integer')
x2 = LpVariable('x2', lowBound = 0, cat = 'Integer')
x3 = LpVariable('x3', lowBound = 0, cat = 'Integer')
x4 = LpVariable('x4', lowBound = 0, cat = 'Integer')
x5 = LpVariable('x5', lowBound = 0, cat = 'Integer')

#pojemnosc plecaka
pojemnosc = 1200

#maksymalne możliwe ilości do wzięcia
quant = [10,7,8,5,6]

#masy jednostkowe produktów
mass = [46, 40, 42, 38, 10]

#ceny jednostkowe produktów
values = [12, 19, 19, 15, 8]

# declare objective
model += values[0]*x1+values[1]*x2+values[2]*x3+values[3]*x4+values[4]*x5

# declare constraints
model += mass[0]*x1+mass[1]*x2+mass[2]*x3+mass[3]*x4+mass[4]*x5 <= pojemnosc

for i in range(1,6):
    model += eval('x{}'.format(i)) <= quant[i-1]

# solve
results = model.solve(solver=solver)

# print results
if LpStatus[results] == 'Optimal': print('The solution is optimal.')
print(f'Objective value: z* = {value(model.objective)}')
print(f'Mass value: z* = {mass[0]*value(x1)+mass[1]*value(x2)+mass[2]*value(x3)+mass[3]*value(x4)+mass[4]*value(x5)}')
print(f'Solution: x1* = {value(x1)}, x2* = {value(x2)}, x3* = {value(x3)}, x4* = {value(x4)}, x5* = {value(x5)}')

The solution is optimal.
Objective value: z* = 492.0
Mass value: z* = 1188.0
Solution: x1* = 7.0, x2* = 7.0, x3* = 8.0, x4* = 5.0, x5* = 6.0


**3. Google OR-Tools**

Podstawy - problem plecakowy w wersji całkowitoliczbowej

In [100]:
!pip install ortools

In [101]:
from ortools.linear_solver import pywraplp

# ustalanie stałych
# wartości przedmiotów
item1value = 100
item2value = 0.5

#pojemność plecaka:
knapsack_cap = 20

#dostępna masa przedmiotów:
item1maxmass = 6.3
item2maxmass = 15

# wywołanie solvera
solver = pywraplp.Solver.CreateSolver('SCIP')

# określenie zmiennych
item1mass = solver.IntVar(0.0, solver.infinity(), 'item1mass')
item2mass = solver.IntVar(0.0, solver.infinity(), 'item2mass')

# określenie celu
solver.Maximize(item1value*item1mass + item2value*item2mass)

# określenie ograniczeń
solver.Add(item1mass + item2mass <= knapsack_cap)
solver.Add(item1mass <= item1maxmass)
solver.Add(item2mass <= item2maxmass)

# rozwiązanie
results = solver.Solve()

# wyświetlanie wyniku
if results == pywraplp.Solver.OPTIMAL: print(f'The solution is optimal.')
print(f'Objective value: z* = {solver.Objective().Value()}')
print(f'Solution: item1mass* = {item1mass.solution_value()}, item2mass* = {item2mass.solution_value()}')

The solution is optimal.
Objective value: z* = 607.0
Solution: item1mass* = 6.0, item2mass* = 14.0


Funkcja do rozwiązywania problemu plecakowego z wieloma plecakami

In [102]:
from ortools.linear_solver import pywraplp

def solving_multiple_knapsacks(itemsValues, itemsMass, itemsQuantities, knapsacksCapacities):
    num_items = len(itemsValues)
    num_knapsacks = len(knapsacksCapacities)

    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver could not be created. Check OR-Tools installation.")
        return None, None

    # Decision variables (avoid excessive memory use)
    itemCounts = [[solver.IntVar(0, min(itemsQuantities[i], knapsacksCapacities[j] // itemsMass[i]), f'item{i}_knapsack{j}')
                   for j in range(num_knapsacks)] for i in range(num_items)]

    # Objective function: Maximize total value
    solver.Maximize(solver.Sum([itemsValues[i] * itemCounts[i][j] for i in range(num_items) for j in range(num_knapsacks)]))

    # Capacity constraints (knapsacks cannot exceed their weight limit)
    for j in range(num_knapsacks):
        solver.Add(solver.Sum([itemsMass[i] * itemCounts[i][j] for i in range(num_items)]) <= knapsacksCapacities[j])

    # Item quantity constraints (each item cannot exceed available stock)
    for i in range(num_items):
        solver.Add(solver.Sum([itemCounts[i][j] for j in range(num_knapsacks)]) <= itemsQuantities[i])

    # Solve the problem
    result_status = solver.Solve()

    if result_status == pywraplp.Solver.OPTIMAL:
        obj_val = solver.Objective().Value()
        solution = [[itemCounts[i][j].solution_value() for j in range(num_knapsacks)] for i in range(num_items)]
        return solution, obj_val
    else:
        return None, None

Przykładowy problem

In [103]:
itemvalues = [10, 20, 30]
itemmass = [2, 3, 4]
itemBounds = [3, 2, 2]
capacities = [5, 7]

a, _ = solving_multiple_knapsacks(itemvalues, itemmass, itemBounds, capacities)

# Wyświetlanie rozwiązania
if a is not None:
    print("Rozwiązanie:", a,  "\n Wartość celu:", _)
else:
    print("Nie znaleziono rozwiązania")

Rozwiązanie: [[1.0, 0.0], [1.0, 1.0], [0.0, 1.0]] 
 Wartość celu: 80.0


Próba odszukaia rozwiązania dla problemu plecakowego znanego nam z zajęć z Uczenia Maszynowego na poprzednim semestrze:

In [ ]:
import pandas as pd
ladybug_capacity = pd.read_csv("csvFiles\Ladybug2.csv", sep= ";", index_col=0)
ladybug_capacity = ladybug_capacity.to_numpy().T.flatten().tolist()

lidl = pd.read_csv("csvFiles\Lidle2.csv", sep= ";", index_col=0, header=None)
weights, net_worths, quantities = lidl.to_numpy().T

weights = weights.tolist()
net_worths = net_worths.tolist()
quantities = quantities.tolist()
a, _ = solving_multiple_knapsacks(net_worths, weights, quantities, ladybug_capacity)

if a is not None:
    print("Rozwiązanie:", a,  "\n Wartość celu:", _)
else:
    print("Nie znaleziono rozwiązania")

**Uwaga**

Google-OR posiada również **bezpośrednie narzędzie** do rozwiązywania problemów plecakowych.

Przykładowy od ze strony Google-OR:

In [ ]:
from ortools.algorithms.python import knapsack_solver


def main():
    # Create the solver.
    solver = knapsack_solver.KnapsackSolver(
        knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        "KnapsackExample",
    )

    values = [
        # fmt:off
      360, 83, 59, 130, 431, 67, 230, 52, 93, 125, 670, 892, 600, 38, 48, 147,
      78, 256, 63, 17, 120, 164, 432, 35, 92, 110, 22, 42, 50, 323, 514, 28,
      87, 73, 78, 15, 26, 78, 210, 36, 85, 189, 274, 43, 33, 10, 19, 389, 276,
      312
        # fmt:on
    ]
    weights = [
        # fmt: off
      [7, 0, 30, 22, 80, 94, 11, 81, 70, 64, 59, 18, 0, 36, 3, 8, 15, 42, 9, 0,
       42, 47, 52, 32, 26, 48, 55, 6, 29, 84, 2, 4, 18, 56, 7, 29, 93, 44, 71,
       3, 86, 66, 31, 65, 0, 79, 20, 65, 52, 13],
        # fmt: on
    ]
    capacities = [850]

    solver.init(values, weights, capacities)
    computed_value = solver.solve()

    packed_items = []
    packed_weights = []
    total_weight = 0
    print("Total value =", computed_value)
    for i in range(len(values)):
        if solver.best_solution_contains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]
    print("Total weight:", total_weight)
    print("Packed items:", packed_items)
    print("Packed_weights:", packed_weights)


if __name__ == "__main__":
    main()